In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from PIL import Image

from pathlib import Path
import random
from tqdm import tqdm

In [ ]:
# shutil.copy('/content/DSC00028.txt','/content/drive/MyDrive')

In [ ]:
source_images = '/content/drive/MyDrive/capstone/workings/Project_LocalMachine/data/data/images_handheld/'
source_annotations = '/content/drive/MyDrive/capstone/workings/Project_LocalMachine/data/annotations_handheld.csv'
destination_images_label = '/content/drive/MyDrive/yolov5_capstone/data/labels/'
destination_images = '/content/drive/MyDrive/yolov5_capstone/data/images/'

In [ ]:
df = pd.read_csv(source_annotations)
df = df[df.columns[:5]]
df.head(5)

,image,x1,y1,x2,y2
0,DSC00025.JPG,1864,2064,2864,1648
1,DSC00026.JPG,2872,1416,3152,2016
2,DSC00027.JPG,0,0,0,0
3,DSC00028.JPG,3248,1540,3572,2048
4,DSC00028.JPG,3444,1916,3852,2116


In [ ]:
image_name_list = next(os.walk(source_images))[2]

In [ ]:
img_list = []

In [ ]:
for filename in tqdm(image_name_list):
  #print(Image.open(source_images+filename).size[0])
  if(Image.open(source_images+filename).size[0] == 6000):
    img_list.append(filename)

100%|██████████| 1787/1787 [07:31<00:00,  3.95it/s]


Images with shape (6000,4000)

In [ ]:
len(img_list)

1618

In [ ]:
def converter(annotation):#,image_shape):
  img_width,img_height = 6000,4000
  #img_width,img_height = img_shape[1],img_shape[0]
  x1,y1,x2,y2 = annotation
  if(x1+y1+x2+y2 == 0):
      return 0
  label = 0 
  label_width = abs(x2-x1)
  label_height = abs(y2-y1)
  x_center = min(x1,x2) + label_width/2
  y_center = img_height -(min(y1,y2) + label_height/2)
  yolo_annotation = [label,x_center/img_width,y_center/img_height
                      ,label_width/img_width,label_height/img_height]
  return yolo_annotation

In [ ]:
def convert_to_yolo(annotation,image_name):
  data_to_write = []
  for annot in annotation:
    conv_annot = converter(annot)
    data_to_write.append(conv_annot)
  
  filename = destination_images_label+image_name.split('.')[0]+".txt"
  if(data_to_write[0] == 0):
    np.savetxt(filename,np.array([]))
  else:
    np.savetxt(filename,np.array(data_to_write),delimiter=',',fmt=' '.join(['%i'] + ['%1.6f']*4))

In [ ]:
#for filename in tqdm(image_name_list):
for filename in tqdm(img_list):
  #print(Image.open(source_images+filename).size[0])
  # if(Image.open(source_images+filename).size[0] == 6000):
  #   count+=1
  df_by_imagename = df.loc[df['image'] == filename]
  df_by_imagename = df_by_imagename.drop(columns='image')
  df_by_imagename = df_by_imagename.to_numpy()
  convert_to_yolo(df_by_imagename,filename)
  
  

100%|██████████| 1618/1618 [00:24<00:00, 66.12it/s]


In [ ]:
# img_list = []

In [ ]:

# for filename in image_name_list:
#   #print(Image.open(source_images+filename).size[0])
#   if(Image.open(source_images+filename).size[0] == 6000):
#     img_list.append(filename)

In [ ]:
len(img_list)

1618

In [ ]:
from tqdm import tqdm
for img in tqdm(img_list):
  shutil.copy(source_images+img,destination_images)

100%|██████████| 1618/1618 [18:46<00:00,  1.44it/s]


[Yolov5 Dataset utils for autosplit](https://github.com/ultralytics/yolov5/blob/0cc7c587870f31f0fc175a74048ceca616870aea/utils/datasets.py#L818-L840)

In [ ]:
IMG_FORMATS = ['bmp', 'JPG', 'jpeg', 'png', 'tif', 'tiff', 'dng', 'webp', 'mpo']

In [ ]:
!python --version

Python 3.7.12


In [ ]:

def autosplit(path='../datasets/coco128/images', weights=(0.9, 0.1, 0.0), annotated_only=False):
    """ Autosplit a dataset into train/val/test splits and save path/autosplit_*.txt files
    Usage: from utils.datasets import *; autosplit()
    Arguments
        path:            Path to images directory
        weights:         Train, val, test weights (list, tuple)
        annotated_only:  Only use images with an annotated txt file
    """
    path = Path(path)  # images dir
    files = sum([list(path.rglob(f"*.{img_ext}")) for img_ext in IMG_FORMATS], [])  # image files only
    n = len(files)  # number of files
    print(n)
    random.seed(0)  # for reproducibility
    indices = random.choices([0, 1, 2], weights=weights, k=n)  # assign each image to a split

    txt = ['autosplit_train.txt', 'autosplit_val.txt', 'autosplit_test.txt']  # 3 txt files

   # [(path.parent / x).unlink()  for x in txt]  # remove existing

    print(f'Autosplitting images from {path}' + ', using *.txt labeled images only' * annotated_only)
    for i, img in tqdm(zip(indices, files), total=n):
        if not annotated_only or Path(img2label_paths([str(img)])[0]).exists():  # check label
            with open(path.parent / txt[i], 'a') as f:
                f.write('./' + img.relative_to(path.parent).as_posix() + '\n')  # add image to txt file

In [ ]:
autosplit('/content/drive/MyDrive/yolov5_capstone/data/images',weights=(0.85,0.1,0.05))

1618
Autosplitting images from /content/drive/MyDrive/yolov5_capstone/data/images


100%|██████████| 1618/1618 [00:10<00:00, 149.49it/s]
